<a href="https://colab.research.google.com/github/mohanrajmit/glaucoma-classification/blob/main/Transfer_Learning_glaucoma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
from pathlib import Path
import numpy as np
import joblib
from keras.preprocessing import image
from keras.applications import xception
import imutils
from imutils import paths

# Empty lists to hold the images and labels for each each image
x_train = []
y_train = []

In [25]:
!git clone https://github.com/mohanrajmit/glaucoma-classification.git

fatal: destination path 'glaucoma-classification' already exists and is not an empty directory.


In [26]:
# Load the training data set by looping over every image file
for image_file in paths.list_images("/content/glaucoma-classification/dataset"):

    # Load the current image file
    image_data = image.load_img(image_file, target_size=(73, 73))

    # Convert the loaded image file to a numpy array
    image_array = image.img_to_array(image_data)

    # Add the current image to our list of training images
    x_train.append(image_array)

    # Add a label for this image. If it was a not_bird image, label it 0. If it was a bird, label it 1.
    path = image_file.split("/")[-2]
    y_train.append(path)

In [28]:
y_train[-1]

'suspicious_glaucoma'

In [29]:
# Convert the list of separate images into a single 4D numpy array. This is what Keras expects.
x_train = np.array(x_train)

# Normalize image data to 0-to-1 range
x_train = xception.preprocess_input(x_train)

# Load the pre-trained neural network to use as a feature extractor
feature_extractor = xception.Xception(weights='imagenet', include_top=False, input_shape=(73, 73, 3))

# Extract features for each image (all in one pass)
features_x = feature_extractor.predict(x_train)

# Save the array of extracted features to a file
joblib.dump(features_x, "x_train.dat")

# Save the matching array of expected values to a file
joblib.dump(y_train, "y_train.dat")


['y_train.dat']

In [35]:
# Load data set of extracted features
x_train = joblib.load("x_train.dat")
y_train = joblib.load("y_train.dat")


In [36]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
y_train=le.fit_transform(y_train)

In [53]:
y_train

array([0, 0, 0, ..., 1, 1, 1])

In [54]:
le.classes_

array(['glaucoma', 'notglaucoma'], dtype='<U11')

In [37]:
y_train[-1]

1

In [38]:
Y_train = np.array(y_train)
print(x_train.shape[1:])
print(Y_train.shape)

(3, 3, 2048)
(4854,)


In [39]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten

In [40]:
# Create a model and add layers
model = Sequential()

# Add layers to our model
model.add(Flatten(input_shape=x_train.shape[1:]))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])


In [41]:
# Train the model
model.fit(x_train,Y_train,epochs=10,shuffle=True, verbose=2)

# Save the trained model to a file so we can use it to make predictions later
model.save("glaucoma_classifier_model.h5")

Epoch 1/10
152/152 - 1s - loss: 0.5691 - accuracy: 0.7734
Epoch 2/10
152/152 - 1s - loss: 0.3875 - accuracy: 0.8218
Epoch 3/10
152/152 - 1s - loss: 0.3678 - accuracy: 0.8321
Epoch 4/10
152/152 - 1s - loss: 0.3489 - accuracy: 0.8412
Epoch 5/10
152/152 - 1s - loss: 0.3417 - accuracy: 0.8430
Epoch 6/10
152/152 - 1s - loss: 0.3286 - accuracy: 0.8488
Epoch 7/10
152/152 - 1s - loss: 0.3198 - accuracy: 0.8609
Epoch 8/10
152/152 - 1s - loss: 0.2940 - accuracy: 0.8712
Epoch 9/10
152/152 - 1s - loss: 0.2834 - accuracy: 0.8723
Epoch 10/10
152/152 - 1s - loss: 0.2858 - accuracy: 0.8714


In [42]:
from keras.models import load_model
from sklearn.metrics import confusion_matrix, classification_report

In [44]:
# Empty lists to hold the images and labels for each each image
x_test = []
y_test = []

# Load the test data set by looping over every image file
for image_file in paths.list_images("/content/glaucoma-classification/test_dataset"):

    # Load the current image file
    image_data = image.load_img(image_file, target_size=(73, 73))

    # Convert the loaded image file to a numpy array
    image_array = image.img_to_array(image_data)

    # Add the current image to our list of test images
    x_test.append(image_array)
    image_path = image_file.split("/")[-2]
    y_test.append(image_path)
    

# Convert the lis_t of test images to a numpy array

x_test = np.array(x_test)

# Normalize test data set to 0-to-1 range
x_test = xception.preprocess_input(x_test)

In [45]:
y_test=le.fit_transform(y_test)

In [51]:
y_test



array([0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1])

In [52]:
le.inverse_transform(y_test)

array(['glaucoma', 'glaucoma', 'glaucoma', 'glaucoma', 'glaucoma',
       'glaucoma', 'glaucoma', 'notglaucoma', 'notglaucoma',
       'notglaucoma', 'notglaucoma', 'notglaucoma', 'notglaucoma'],
      dtype='<U11')

In [47]:
# Load our trained classifier model
model = load_model("/content/glaucoma_classifier_model.h5")

# Extract features for each image (all in one pass)
features_x = feature_extractor.predict(x_test)

# Given the extracted features, make a final prediction using our own model
predictions = model.predict(features_x)

# If the model is more than 50% sure the object is a bird, call it a bird.
# Otherwise, call it "not a bird".
predictions = predictions > 0.5

# Calculate how many mis-classifications the model makes
tn, fp, fn, tp = confusion_matrix(y_test, predictions).ravel()
print(f"True Positives: {tp}")
print(f"True Negatives: {tn}")
print(f"False Positives: {fp}")
print(f"False Negatives: {fn}")

# Calculate Precision and Recall for each class
report = classification_report(y_test, predictions)
print(report)

True Positives: 2
True Negatives: 0
False Positives: 7
False Negatives: 4
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         7
           1       0.22      0.33      0.27         6

    accuracy                           0.15        13
   macro avg       0.11      0.17      0.13        13
weighted avg       0.10      0.15      0.12        13

